In [ ]:
import json
import time

import numpy as np
import seaborn as sn
from sklearn.metrics import ConfusionMatrixDisplay

def classify_subcategory(text):
    likelihoods = [0 for _ in range(len(SUBCATEGORIES))]
    text = set(removePunctuation(text).lower().split())
    for token in text:
        if token in stop_words:
            continue

        if token in word_to_subcat:
            for i in range(len(SUBCATEGORIES)):
                likelihoods[i] += word_to_subcat[token][i]
        else:
            print('Token not in word_to_subcat:', token)

    return SUBCATEGORIES[likelihoods.index(max(likelihoods))]


def hhi(arr):
    return sum([_**2 for _ in arr]) / sum(arr) ** 2


def normalized_hhi(arr):
    return (hhi(arr) - 1/len(arr)) / (1 - 1/len(arr))


def removePunctuation(s, punctuation='''.,!-;:'"\/?@#$%^&*_~()[]{}“”‘’'''):
    return ''.join(ch for ch in s if ch not in punctuation)


with open('../stop-words.txt') as f:
    stop_words = set(f.readlines())
    stop_words = set([word.strip() for word in stop_words])

with open('../subcategories.txt') as f:
    SUBCATEGORIES = [line.strip() for line in f.readlines()]

time_now = time.perf_counter()
with open('word_to_subcat_normalized.json') as f:
    word_to_subcat = json.load(f)
print('Time to load word_to_subcat:', time.perf_counter() - time_now)

for word in word_to_subcat:
    if normalized_hhi(word_to_subcat[word]) < 0.01:
        print(word)
    # factor = normalized_hhi(word_to_subcat[word])
    factor = normalized_hhi(
        word_to_subcat[word])**5 / sum(word_to_subcat[word])
    word_to_subcat[word] = [factor * i for i in word_to_subcat[word]]


print('=====Begin Testing=====')
time_now = time.perf_counter()

questions = open('questions.json')
counter = 0
correct = 0
confusion_matrix = [[0 for _ in range(len(SUBCATEGORIES))]
                    for _ in range(len(SUBCATEGORIES))]
for line in questions:
    data = json.loads(line)
    if 'subcategory' not in data:
        continue

    counter += 1
    subcategory = data['subcategory']
    if subcategory not in SUBCATEGORIES:
        continue

    if data['type'] == 'tossup':
        prediction = classify_subcategory(data['question'])
    if data['type'] == 'bonus' and 'parts' in data and len(data['parts']) == 3:
        prediction = classify_subcategory(
            data['leadin'] + ' ' + data['parts'][0] + data['parts'][1] + data['parts'][2])

    subcategory_index = SUBCATEGORIES.index(subcategory)
    prediction_index = SUBCATEGORIES.index(prediction)

    confusion_matrix[subcategory_index][prediction_index] += 1

    if subcategory == prediction:
        correct += 1

    if counter % 10000 == 0:
        print(f'{correct}/{counter}')

print('=====End Testing=====')
print(f'Accuracy: {correct / counter} ({correct}/{counter})')
print('Time:', time.perf_counter() - time_now, 'seconds')
print('Confusion Matrix:')
for row in confusion_matrix:
    print(row)


In [ ]:
sn.heatmap(confusion_matrix, cmap='Greens', annot=False)

In [ ]:
# ConfusionMatrixDisplay(np.array(confusion_matrix), display_labels='').plot()